In [1]:
# from tqdm import tqdm
from tqdm.notebook import tqdm

import json 
import pandas as pd
import polars as pl
import unicodedata

import polars as pl
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sentence_transformers import SentenceTransformer

import matplotlib.pyplot as plt

In [2]:
class StarRatingDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx], dtype=torch.float32)
        # Convert label to a one-hot vector: label in {1,...,5} -> one-hot of length 5
        y_onehot = torch.zeros(5)
        y_onehot[self.y[idx]-1] = 1.0
        return x, y_onehot

In [3]:
# train_dataset = torch.load('../data/2_training_ready/embedding00/train_dataset00.pth')
# test_dataset = torch.load('../data/2_training_ready/embedding00/test_dataset00.pth')
# val_dataset = torch.load('../data/2_training_ready/embedding00/val_dataset00.pth')
test_dataset = torch.load('../data/2_training_ready/embedding00/test_dataset01.pth')
val_dataset = torch.load('../data/2_training_ready/embedding00/val_dataset01.pth')

/tmp/ipykernel_15150/3211474504.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_dataset = torch.load('../data/2_training_ready/embedding00/test_dataset01.pth')
/tmp

In [4]:
training1 = torch.load("../data/2_training_ready/embedding00/train_dataset01.pth")
training2 = torch.load("../data/2_training_ready/embedding00/train_dataset02.pth")
training3 = torch.load("../data/2_training_ready/embedding00/train_dataset03.pth")
training4 = torch.load("../data/2_training_ready/embedding00/train_dataset04.pth")

# # Combine the embeddings and labels
# combined_embeddings = torch.cat([training1.X, training2.X, training3.X,training4.X], dim=0)
# combined_labels = torch.cat([training1.y, training2.y, training3.y,training4.y], dim=0)
# # 
# # # Create a new EmbeddingDataset with the combined data
# train_dataset = StarRatingDataset(combined_embeddings.numpy(), combined_labels.numpy())

/tmp/ipykernel_15150/3878769508.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  training1 = torch.load("../data/2_training_ready/embedding00/train_dataset01.pth")
/tmp/i

In [5]:
# Concatenate all X values
combined_X = np.concatenate([
    training1.X,
    training2.X,
    training3.X,
    training4.X
], axis=0)

# Concatenate all y values
combined_y = np.concatenate([
    training1.y,
    training2.y,
    training3.y,
    training4.y
], axis=0)

# Create a new dataset with the combined data
train_dataset = StarRatingDataset(combined_X, combined_y)

In [6]:
_X_, _y_ = train_dataset[0]  # Get the first sample
print(f"Shape of X: {_X_.shape}")
print(f"Shape of y: {_y_.shape if isinstance(_y_, torch.Tensor) else type(_y_)}")

Shape of X: torch.Size([1024])
Shape of y: torch.Size([5])


# PORT CODE FROM 4a AND REPURPOSE

In [7]:
test_dataset.y = torch.tensor(test_dataset.y).long()  # Convert to tensor and then to long
train_dataset.y = torch.tensor(train_dataset.y).long()  # Convert to tensor and then to long
val_dataset.y = torch.tensor(val_dataset.y).long()  # Convert to tensor and then to long

In [8]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim=1_024, hidden_dim_1=512,hidden_dim_2=256, output_dim=5, dropout=0.1):
        super(SimpleNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim_1),
            nn.LayerNorm(hidden_dim_1),
            nn.LeakyReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim_1, hidden_dim_2),
            nn.LayerNorm(hidden_dim_2),
            nn.LeakyReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim_2, output_dim)
        )

    def forward(self, x):
        return self.model(x)


NameError: name 'model' is not defined

In [ ]:
# class SimpleNN(nn.Module):
#     def __init__(self, input_dim=1_024, hidden_dim=512, output_dim=5, dropout=0.1):
#         super(SimpleNN, self).__init__()
#         self.model = nn.Sequential(
#             nn.Linear(input_dim, hidden_dim),
#             nn.LayerNorm(hidden_dim),
#             nn.LeakyReLU(),
#             nn.Dropout(dropout),
#             nn.Linear(hidden_dim, output_dim)
#         )
# 
#     def forward(self, x):
#         return self.model(x)


In [10]:
# Set up your dataset and dataloaders
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# # Convert dataset labels to integers (class indices)
# train_dataset.y = train_dataset.y.long()  # Keep 1-based indexing for star ratings
# test_dataset.y = test_dataset.y.long()  # Keep 1-based indexing for star ratings
# val_dataset.y = val_dataset.y.long()

In [ ]:
# Define model, loss function, and optimizer
# input_dim = train_dataset.X.shape[1]  # Number of features in the embeddings
# output_dim = len(torch.unique(train_dataset.y))  # Number of classes

In [11]:
model = SimpleNN().to("cuda")
criterion = nn.CrossEntropyLoss()

In [12]:

total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

Total number of parameters: 658949


In [ ]:
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.631)

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.126)

In [ ]:
# optimizer = optim.AdamW(model.parameters(), lr=1e-4)

In [ ]:
# Training and evaluation loops
def evaluate(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X, y in data_loader:
            X = X.to("cuda")
            y = y.to("cuda")
            outputs = model(X)
            _, predicted = torch.max(outputs, 1)
            true_classes = torch.max(y, 1)[1]  # Convert one-hot to class indices
            correct += (predicted == true_classes).sum().item()  # Adjust prediction for 1-based indexing
            total += y.size(0)
    accuracy = correct / total
    return accuracy

In [ ]:
# num_epochs = 10
num_epochs = 3
# log_interval = 1_500
# log_interval = 10_000
log_interval = 500

In [ ]:
step = 0

train_losses = []
eval_accuracies = []
learning_rates = []  # New list to track learning rates

for epoch in range(num_epochs):
    current_lr = optimizer.param_groups[0]['lr']
    learning_rates.append((epoch, current_lr))
    # learning_rates.append(current_lr)
    
    model.train()
    pbar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}")
    running_loss = 0.0

    for X, y in pbar:
        X = X.to("cuda")
        y = y.to("cuda")
        # Forward pass
        outputs = model(X)
        # loss = criterion(outputs, y - 1)  # Shift labels for 0-based indexing during training
        loss = criterion(outputs, y)  # Shift labels for 0-based indexing during training
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

        running_loss += loss.item()
        step += 1

        # Log progress every log_interval steps
        if step % log_interval == 0:
            eval_accuracy = evaluate(model, test_loader)
            eval_accuracies.append((step, eval_accuracy))
            print(f"Step {step}, Loss: {loss.item():.4f}, Eval Accuracy: {eval_accuracy:.4f}")

    train_losses.append(running_loss / len(train_loader))
    scheduler.step()



In [ ]:
# Plot training loss and evaluation accuracy
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Training Loss")

plt.subplot(1, 2, 2)
steps, accuracies = zip(*eval_accuracies)
plt.plot(steps, accuracies, label="Evaluation Accuracy")
plt.xlabel("Step")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Evaluation Accuracy")

plt.tight_layout()
plt.show()


In [ ]:
epochs, lrs = zip(*learning_rates)
plt.figure(figsize=(10, 5))
plt.plot(epochs, lrs, 'b-')
plt.yscale('log')
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.title('Learning Rate Schedule')
plt.grid(True)
plt.show()

In [ ]:
predictions = []
correct_values = []
model.eval()
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

with torch.no_grad():
    for batch_x, batch_y in tqdm(val_loader, desc="Hit Rate Eval", unit="batch"):
        batch_y = torch.argmax(batch_y, dim=1)

        batch_x = batch_x.to("cuda")
        batch_y = batch_y.to("cuda")

        pred = model(batch_x)

        # Compute hit rate
        y_pred_classes = torch.argmax(torch.softmax(pred, dim=1), dim=1)
        # y_true_classes = torch.argmax(batch_y, dim=1) # TK TK ADD BACK FOR EMD

        for i in [i.item() for i in torch.argmax(torch.softmax(pred, dim=1), dim=1)]:
            predictions.append(i)
        for i in [i.item() for i in batch_y]: # torch.argmax(batch_y, dim=1):
            correct_values.append(i)

In [ ]:
df = pl.DataFrame({"true": correct_values, "predicted": predictions})
df.write_csv("../output/predictions/embedding_BIG.csv")